In [142]:
import sys
import collections
import bt2
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import bokeh
import itertools

In [143]:
stock_trace_path = "/home/wagle/.ros/tracing/vanilla-ros-graph-gen-20241020-170127/"
modified_trace_path = "/home/wagle/.ros/tracing/modified-ros-graph-gen-20241021-020603/"

In [144]:
def get_callback_invocation_data(iterator):
    invocations = {}
    for msg in iterator:
        # We only care about event messages
        if type(msg) is not bt2._EventMessageConst:
            continue
    
        # Event of the event message
        event = msg.event
    
        # Keep only `unc_ros_graph_generator` events
        if 'unc_ros_graph_generator:' not in event.cls.name:
            continue
        if 'unc_ros_graph_generator:subscribe' in event.cls.name:
            continue
    
        invocation_count = str(event.payload_field['invocation_count'])
        # Check if the key exists; if not, create a new list
        if invocation_count not in invocations:
            invocations[invocation_count] = []
    
        # Insert the new element into the list
        invocations[invocation_count].append(msg)

    return invocations

In [145]:
stock_it = bt2.TraceCollectionMessageIterator(stock_trace_path)
stock_ros_chain_invocations = get_callback_invocation_data(stock_it)
print("In the stock-ROS data, there were {} invocations of the source node.".format(len(stock_ros_chain_invocations)))

modified_it = bt2.TraceCollectionMessageIterator(modified_trace_path)
modified_ros_chain_invocations = get_callback_invocation_data(modified_it)
print("In the modified-ROS data, there were {} invocations of the source node.".format(len(modified_ros_chain_invocations)))



In the stock-ROS data, there were 1513 invocations of the source node.
In the modified-ROS data, there were 1757 invocations of the source node.


In [146]:
def get_data_numpy(invocations):

    # Keeps track of the source callbacks' names and start times per invocation
    sources = {"source" : None }
    
    # Keeps track of the leaf callbacks' names and end times per invocation
    leaves = {"N1_CB_3": None, "N2_CB_3": None}
    
    num_invocations = len(invocations.keys())
    num_leaves = len(leaves.keys())
    num_sources = len(sources.keys())
    
    # invocation #, source callback name, source start timestamp (ns), leaf callback name, leaf end timestamp (ns), response time (ns)
    num_fields = 5
    invocations_numpy = np.ones((num_invocations * num_sources * num_leaves, 6), dtype=object)
    curr_row = 0
    
    for invocation_num in invocations.keys():
        for msg in invocations[invocation_num]:
            event_type = msg.event.cls.name
            name = ""
            if 'callback_' in event_type:
                name = msg.event.payload_field['callback_name']
    
                if 'callback_begin' in event_type and name in sources: 
                    sources[name] = msg.default_clock_snapshot.ns_from_origin
    
                elif 'callback_end' in event_type and name in leaves:
                    leaves[name] = msg.default_clock_snapshot.ns_from_origin
                    for source in sources:
                        start_ts = sources[source]
                        end_ts = leaves[name]
                        assert(start_ts != None and end_ts != None)
                        assert(end_ts >= start_ts)
                    
                        resp_time = end_ts - start_ts
                        invocations_numpy[curr_row] = np.array([int(invocation_num), source, start_ts, str(name), end_ts, resp_time], dtype=object)
                        curr_row = curr_row + 1
                    
            # if 'publish' in event_type:
            #     name = msg.event.payload_field['sender_callback_name']
            #     topic = msg.event.payload_field['topic_name']

    return invocations_numpy

In [147]:
stock_ros_numpy = get_data_numpy(stock_ros_chain_invocations)
print("Shape of stock-ROS numpy array: {}".format(stock_ros_numpy.shape))
modified_ros_numpy = get_data_numpy(modified_ros_chain_invocations)
print("Shape of modified-ROS numpy array: {}".format(modified_ros_numpy.shape))

Shape of stock-ROS numpy array: (3026, 6)
Shape of modified-ROS numpy array: (3514, 6)


In [148]:
column_names = ['Invocation #', 'Source', 'Source start time (ns)', 'Leaf', 'Leaf end time (ns)', 'Chain Resp Time (ns)']

stock_ros_pandas = pd.DataFrame(stock_ros_numpy, columns=column_names)
stock_ros_pandas = stock_ros_pandas.astype({
    'Invocation #': 'int64',
    'Source': 'object',
    'Source start time (ns)': 'int64',
    'Leaf': 'object',
    'Leaf end time (ns)': 'int64',
    'Chain Resp Time (ns)': 'int64'
})

modified_ros_pandas = pd.DataFrame(modified_ros_numpy, columns=column_names)
modified_ros_pandas = modified_ros_pandas.astype({
    'Invocation #': 'int64',
    'Source': 'object',
    'Source start time (ns)': 'int64',
    'Leaf': 'object',
    'Leaf end time (ns)': 'int64',
    'Chain Resp Time (ns)': 'int64'
})

In [161]:
# Enable Bokeh in Jupyter Notebook (if using Jupyter)
output_notebook()
def plot_chain_resp_times(raw_pandas_data, source, leaf, stock_or_mod):

    colors = ['#29788E', '#DD4968', '#410967']
    color_idx = 0

    title = "({} ROS Jazzy) Response time (ns) of chain `{}` --> `{}`".format(stock_or_mod, source,leaf)
    p = figure(title=title, x_axis_label='Chain Invocation #', y_axis_label='Chain Reponse Time (ms)', y_range=(0,5.0), x_range = (-20.0,1800.0))
    p.width = 1100
    p.height = 800
    p.title.text_font_size = '20pt'       # Title size
    p.xaxis.axis_label_text_font_size = '16pt'  # X-axis label size
    p.yaxis.axis_label_text_font_size = '16pt'  # Y-axis label size
    p.xaxis.major_label_text_font_size = '12pt'  # X-axis tick labels size
    p.yaxis.major_label_text_font_size = '12pt'  # Y-axis tick labels size

    
    filtered_pandas_data = raw_pandas_data[(raw_pandas_data['Source'] == source) & (raw_pandas_data['Leaf'] == leaf)]
    resp_times_ms = filtered_pandas_data['Chain Resp Time (ns)']/1000000
    resp_time_mov_avg = resp_times_ms.rolling(window=10).mean()
    
    p.line(filtered_pandas_data['Invocation #'], resp_time_mov_avg, line_width=2, legend_label="Moving Avg (Window Size = 10)", color=colors[color_idx])
    color_idx = (color_idx + 1) % len(colors)
    p.scatter(filtered_pandas_data['Invocation #'].iloc[::10], resp_times_ms.iloc[::10], size=10, marker="circle", fill_color=colors[color_idx])

    # Show the plot
    show(p)

Loading BokehJS ...

In [162]:
source_leaf_pairs = list(itertools.product(sources, leaves))

for pair in source_leaf_pairs:
    stock_ros_pandas_filtered = stock_ros_pandas[(stock_ros_pandas['Source'] == pair[0]) & (stock_ros_pandas['Leaf'] == pair[1])]
    stock_ros_mean_resp_time = stock_ros_pandas_filtered['Chain Resp Time (ns)'].mean()
    print("Stock ROS ({} --> {}) mean response time = {:.3f} ns = {:.3f} ms".format(pair[0], pair[1], stock_ros_mean_resp_time, stock_ros_mean_resp_time/1000000))
    plot_chain_resp_times(stock_ros_pandas, pair[0], pair[1], "Stock")
    
    modified_ros_pandas_filtered = modified_ros_pandas[(modified_ros_pandas['Source'] == pair[0]) & (modified_ros_pandas['Leaf'] == pair[1])]
    modified_ros_mean_resp_time = modified_ros_pandas_filtered['Chain Resp Time (ns)'].mean()
    print("Modified ROS ({} --> {}) mean response time = {:.3f} ns = {:.3f} ms".format(pair[0], pair[1], modified_ros_mean_resp_time, modified_ros_mean_resp_time/1000000))
    plot_chain_resp_times(modified_ros_pandas, pair[0], pair[1], "Modified")

Stock ROS (source --> N1_CB_3) mean response time = 3486376.719 ns = 3.486 ms


Modified ROS (source --> N1_CB_3) mean response time = 1935105.002 ns = 1.935 ms


Stock ROS (source --> N2_CB_3) mean response time = 3908774.083 ns = 3.909 ms


Modified ROS (source --> N2_CB_3) mean response time = 2128359.769 ns = 2.128 ms


In [173]:
def plot_multiple_chain_resp_times(chain1_raw_pandas_data, chain2_raw_pandas_data, source, leaf):

    colors = ['#d5e8d4', '#6d8fbf', '#f8cecc', '#8660b4']
    color_idx = 0

    title = "(Stock vs Mod. ROS Jazzy) Resp. time (ns) of chain `{}` --> `{}`".format(source,leaf)
    p = figure(title=title, x_axis_label='Chain Invocation #', y_axis_label='Chain Reponse Time (ms)', y_range=(0,5.0), x_range = (-20.0,1800.0))
    p.width = 1100
    p.height = 800
    p.title.text_font_size = '20pt'       # Title size
    p.xaxis.axis_label_text_font_size = '16pt'  # X-axis label size
    p.yaxis.axis_label_text_font_size = '16pt'  # Y-axis label size
    p.xaxis.major_label_text_font_size = '12pt'  # X-axis tick labels size
    p.yaxis.major_label_text_font_size = '12pt'  # Y-axis tick labels size

    
    chain1_filtered_pandas_data = chain1_raw_pandas_data[(chain1_raw_pandas_data['Source'] == source) & (chain1_raw_pandas_data['Leaf'] == leaf)]
    chain1_resp_times_ms = chain1_filtered_pandas_data['Chain Resp Time (ns)']/1000000
    chain1_resp_time_mov_avg = chain1_resp_times_ms.rolling(window=10).mean()

    chain2_filtered_pandas_data = chain2_raw_pandas_data[(chain2_raw_pandas_data['Source'] == source) & (chain2_raw_pandas_data['Leaf'] == leaf)]
    chain2_resp_times_ms = chain2_filtered_pandas_data['Chain Resp Time (ns)']/1000000
    chain2_resp_time_mov_avg = chain2_resp_times_ms.rolling(window=10).mean()

    legend_label = "Stock ROS {} --> {} moving avg".format(source, leaf)
    p.line(chain1_filtered_pandas_data['Invocation #'], chain1_resp_time_mov_avg, line_width=2, legend_label=legend_label, color=colors[0])
    p.scatter(chain1_filtered_pandas_data['Invocation #'].iloc[::10], chain1_resp_times_ms.iloc[::10], size=10, marker="circle", fill_color=colors[1])

    legend_label = "Modified ROS {} --> {} moving avg".format(source, leaf)
    p.line(chain2_filtered_pandas_data['Invocation #'], chain2_resp_time_mov_avg, line_width=2, legend_label=legend_label, color=colors[2])
    p.scatter(chain2_filtered_pandas_data['Invocation #'].iloc[::10], chain2_resp_times_ms.iloc[::10], size=10, marker="circle", fill_color=colors[3])

    # Show the plot
    show(p)

In [174]:
source_leaf_pairs = list(itertools.product(sources, leaves))

for pair in source_leaf_pairs:
    plot_multiple_chain_resp_times(stock_ros_pandas, modified_ros_pandas, pair[0], pair[1])
